# **Adaboost implementation for binary classification**

In [151]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [152]:
# Tạo dữ liệu kiểm thử
X, y = make_classification(n_samples= 1000, n_features = 20, random_state = 42)

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [148]:
class Adaboost:
  def __init__(self,n_estimator,criterion):
    self.n_estimator = n_estimator # Số Stump định xây dựng
    self.criterion = criterion # Độ đo sử dụng để xây dựng stump
    self.Amount_of_says = [] # list Amount of say
    self.result_tests = [] # Kết quả đánh giá stump trên tập training khi xây dựng xong
    self.weights = [] # Danh sách các trọng số của từng tập dữ liệu huấn luyện stump
    self._estimator = [] # Danh sách các mô hình khi đã huấn luyện xong

  def Compute_amount_of_say(self,total_Error):
    amount_of_say = 0.5*np.log((1-total_Error)/total_Error)
    return amount_of_say

  def Compute_weight(self,amount_of_say,old_weight,result_test):
    result_test = np.where(result_test==0,-1,result_test) # Nếu sample phân loại đúng trả 0 -> chuyển về -1: để giảm weight_sample
    weight = old_weight*np.exp(result_test*amount_of_say)
    return weight

  # Hàm huấn luyện mô hình
  def fit(self,X_train,y_train):
    self.X_train = X_train
    self.y_train = y_train
    for estimator in range(self.n_estimator):
      if not estimator:
        weight = np.ones(self.y_train.shape[0])*(1/self.y_train.shape[0])
      else:
        weight = self.Compute_weight(self.Amount_of_says[estimator-1],self.weights[estimator-1],self.result_tests[estimator-1])
      self.weights.append(weight)
      clf = DecisionTreeClassifier(criterion=self.criterion,max_depth=1)
      clf.fit(self.X_train,self.y_train,sample_weight=weight)
      self._estimator.append(clf)
      # kết quả của stump khi đánh giá trên bộ training
      y_pred_train = clf.predict(self.X_train)
      # So sánh kết quả của Stump và kết quả thực tế của trainingset
      result_test = np.not_equal(y_pred_train,self.y_train)# Bằng 1 nếu kết quả khác nhau, = 0 nếu kết quả giống nhau
      total_Error = np.sum(weight[result_test]) # Tính tổng số lỗi
      amount_of_say = self.Compute_amount_of_say(total_Error)
      self.Amount_of_says.append(amount_of_say)
      self.result_tests.append(result_test)

  # Hàm dự đoán mô hình adaboostClassifier
  def predict(self,X_test):
    predict_matrix = []
    self.X_test = X_test
    for clf in self._estimator:
      y_pred = clf.predict(X_test)
      predict_matrix.append(y_pred)
    predict_matrix = np.array(predict_matrix)
    weight_model = np.array(self.Amount_of_says).reshape(-1,1)
    predict_matrix_weight = np.array([
        np.sum((1-predict_matrix)*weight_model,axis= 0),
        np.sum(predict_matrix*weight_model,axis=0)
    ])
    y_pred = np.argmax(predict_matrix_weight,axis=0)
    return y_pred

In [154]:
clf = Adaboost(n_estimator=50,criterion='gini')

In [155]:
clf.fit(X_train,y_train)

In [156]:
y_pred = clf.predict(X_test)

In [157]:
# Đánh giá độ chính xác cảu mô hình
print(f"Độ chính xác của mô hình: {accuracy_score(y_pred,y_test)}")

Độ chính xác của mô hình: 0.86


# **Using Adaboost by sklearn**

In [159]:
from sklearn.ensemble import AdaBoostClassifier

In [160]:
ada = AdaBoostClassifier(n_estimators=50)

In [161]:
ada.fit(X_train,y_train)

AdaBoostClassifier()

In [162]:
y_pred = ada.predict(X_test)
y_pred

array([1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1])

In [163]:
print(f"Độ chính xác của mô hình: {accuracy_score(y_pred,y_test)}")

Độ chính xác của mô hình: 0.875
